In [ ]:
spark.sql("CREATE SCHEMA IF NOT EXISTS development.gold")

# Analysis 1: Mean and Standard Deviation of annual US population (years 2013–2018)

In [ ]:
spark.sql("""
    CREATE OR REPLACE VIEW development.gold.population_stats AS
    SELECT
        AVG(population) AS mean_population,
        STDDEV(population) AS stddev_population
    FROM development.silver.datausa_population
    WHERE year BETWEEN 2013 AND 2018
""")
spark.table("development.gold.population_stats").show()

# Analysis 2: Best year per series_id (year with max sum of value across quarters)

In [ ]:
spark.sql("""
    CREATE OR REPLACE VIEW development.gold.best_year_per_series AS
    WITH yearly AS (
        SELECT series_id, year, SUM(value) AS value
        FROM development.silver.bls_pr_data
        GROUP BY series_id, year
    )
    SELECT series_id, year, value
    FROM (
        SELECT series_id, year, value,
               ROW_NUMBER() OVER (PARTITION BY series_id ORDER BY value DESC) AS rn
        FROM yearly
    ) t
    WHERE rn = 1
""")
spark.table("development.gold.best_year_per_series").orderBy("series_id").show(20)

# Analysis 3: series_id = PRS30006032, period = Q01, with population for that year

In [ ]:
spark.sql("""
    CREATE OR REPLACE VIEW development.gold.series_population_report AS
    SELECT t.series_id, t.year, t.period, t.value, p.population AS population
    FROM development.silver.bls_pr_data t
    LEFT JOIN development.silver.datausa_population p ON t.year = p.year
    WHERE t.series_id = 'PRS30006032' AND t.period = 'Q01'
""")
spark.table("development.gold.series_population_report").orderBy("year").display()